In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
COLUMNS = ["age", "workclass", "edu_level",
           "marital_status", "occupation", "relationship",
           "race", "sex", "hours_per_week",
           "native_country", "income"]

train_df = pd.read_csv(
    filepath_or_buffer="https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    names=COLUMNS,
    engine='python',
    usecols=[0, 1, 4, 5, 6, 7, 8, 9, 12, 13, 14],
    sep=r'\s*,\s*',
    na_values="?"
)

test_df = pd.read_csv(
    filepath_or_buffer="https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
    names=COLUMNS,
    skiprows=[0],
    engine='python',
    usecols=[0, 1, 4, 5, 6, 7, 8, 9, 12, 13, 14],
    sep=r'\s*,\s*',
    na_values="?"
)


In [3]:
# Drop rows with missing values
train_df = train_df.dropna(how="any", axis=0)
test_df = test_df.dropna(how="any", axis=0)

# To reduce the complexity, we binarize the attribute
# To reduce the complexity, we binarize the attribute


def mapping(tuple):
    # age, 37
    tuple['age'] = 1 if tuple['age'] > 37 else 0
    # workclass
    tuple['workclass'] = 0 if tuple['workclass'] != 'Private' else 1
    # edu-level
    tuple['edu_level'] = 1 if tuple['edu_level'] > 9 else 0
    # maritial statue
    tuple['marital_status'] = 1 if tuple['marital_status'] == "Married-civ-spouse" else 0
    # occupation
    tuple['occupation'] = 1 if tuple['occupation'] == "Craft-repair" else 0
    # relationship
    tuple['relationship'] = 0 if tuple['relationship'] == "Not-in-family" else 1
    # race
    tuple['race'] = 0 if tuple['race'] != "White" else 1
    # sex
    tuple['sex'] = 0 if tuple['sex'] != "Male" else 1
    # hours per week
    tuple['hours_per_week'] = 1 if tuple['hours_per_week'] > 40 else 0
    # native country
    tuple['native_country'] = 1 if tuple['native_country'] == "United-States" else 0
    # income
    tuple['income'] = 1 if tuple['income'] == '>50K' or tuple['income'] == '>50K.' else 0
    return tuple


train_df = train_df.apply(mapping, axis=1)
test_df = test_df.apply(mapping, axis=1)

In [266]:
train_df[40:100]

,age,workclass,edu_level,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,income
43,1,1,0,0,0,1,1,0,0,1,0
44,0,1,0,0,0,0,1,1,0,1,0
45,1,0,1,1,0,1,0,1,0,1,1
46,1,1,0,1,0,1,1,1,0,1,0
47,1,1,1,0,0,1,1,0,0,1,0
48,1,0,1,1,1,1,1,1,0,1,0
49,0,1,1,0,0,0,1,1,1,1,0
50,0,1,1,1,0,1,0,0,0,1,0
52,1,1,1,1,0,1,1,0,1,0,1
53,1,0,1,0,0,0,1,1,1,1,1


In [5]:
train_data = torch.from_numpy(train_df.values)
test_data = torch.from_numpy(test_df.values)
# merge two datasets
dataset = torch.cat((train_data,test_data), 0)

In [6]:
from torch.utils.data import Dataset,DataLoader
class AdultDataset(Dataset):
    def __init__(self, data_set):
        self.x = data_set
        self.len = data_set.size()[0]
    def __getitem__(self,index):
        return self.x[index]
    def __len__(self):
        return self.len
adultDataset = AdultDataset(dataset)
dataLoader = DataLoader(dataset=adultDataset, batch_size=128, shuffle=True)



In [149]:
# a basic Generator


class Generator(nn.Module):
    def __init__(self, f, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size*2)
        self.map2 = nn.Linear(hidden_size*2, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        # f is action function
        self.f = f

    def forward(self, x):
        x = self.map1(x)
        x = self.f(x)
        x = self.map2(x)
        x = self.f(x)
        x = self.map3(x)
        x = self.f(x)
        return x

# a basic Discriminator

In [150]:
class Discriminator(nn.Module):
    def __init__(self, f, input_size, hidden_size, output_size):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size*2)
        self.map2 = nn.Linear(hidden_size*2, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        # f is action function
        self.f = f

    def forward(self, x):
        x = self.f(self.map1(x))
        x = self.f(self.map2(x))
        return torch.sigmoid(self.map3(x))

In [271]:
class CFGAN(nn.Module):
    def __init__(self, f):
        super(CFGAN, self).__init__()

        self.age_net = Generator(
            f, 1, 4, 1)
        self.workclass_net = Generator(
            f, 5, 16, 1)
        self.edu_level_net = Generator(
            f, 6, 16, 1)
        self.marital_status_net = Generator(
            f, 5, 16, 1)
        self.occupation_net = Generator(
            f, 6, 16, 1)
        self.relationship_net = Generator(
            f, 6, 16, 1)
        self.race_net = Generator(
            f, 1, 4, 1)
        self.sex_net = Generator(
            f, 1, 4, 1)
        self.hours_per_week_net = Generator(
            f, 7, 16, 1)
        self.native_country_net = Generator(
            f, 1, 4, 1)
        self.income_net = Generator(
            f, 11, 32, 1)

    def forward(self, input, intervention=-1):
        name = ["age","workclass","edu_level","marital_status","occupation","relationship","race","sex","hours_per_week","native_country","income"]
        Z = dict(zip(name, input.transpose(0, 1).view(len(name), -1, 1)))

        # hight = 0 in the graph
        # sex should considered about intervention
        if(intervention == -1):
            self.sex = self.race_net(Z["sex"])
        elif(intervention == 0):
            self.sex = torch.zeros(Z["sex"].size())
        else:
            self.sex = torch.ones(Z["sex"].size())
        self.age = self.age_net(Z["age"])
        self.race = self.sex_net(Z["race"])
        self.native_country = self.native_country_net(Z["native_country"])

        # hight = 1 in the graph
        self.marital_status = self.marital_status_net(torch.cat(
            [Z["marital_status"], self.race, self.age,
                self.sex, self.native_country], 1
        ))

        # hight = 2 in the gragh
        self.edu_level = self.edu_level_net(torch.cat(
            [Z["edu_level"], self.race, self.age, self.sex,
                self.native_country, self.marital_status], 1
        ))

        # hight = 3 in the gragh
        self.occupation = self.occupation_net(torch.cat(
            [Z["occupation"], self.race, self.age, self.sex,
                self.marital_status, self.edu_level], 1
        ))

        self.hours_per_week = self.hours_per_week_net(torch.cat(
            [Z["hours_per_week"], self.race, self.age, self.sex,
             self.native_country, self.marital_status, self.edu_level], 1
        ))

        self.workclass = self.workclass_net(torch.cat(
            [Z["workclass"], self.age, self.marital_status,
                self.edu_level, self.native_country], 1
        ))

        self.relationship = self.relationship_net(torch.cat(
            [Z["relationship"], self.age, self.sex, self.native_country,
                self.marital_status, self.edu_level], 1
        ))

        # hight = 4 in the gragh

        self.income = self.income_net(torch.cat(
            [Z["income"], self.race, self.age, self.sex, self.native_country, self.marital_status,
                self.edu_level, self.occupation, self.hours_per_week, self.workclass, self.relationship], 1
        ))

        return torch.cat([self.age, self.workclass, self.edu_level, self.marital_status,
        self.occupation, self.relationship, self.race, self.sex,
        self.hours_per_week, self.native_country, self.income], 1)

In [274]:
num_epochs = 100
g_steps = 5
g2_steps = 50
batch = 256
LR = 0.0005
print_interval = 1
# action function
discriminator_activation_function = nn.LeakyReLU(0.2)
generator_activation_function = torch.tanh

# net init
discriminator_1 = Discriminator(
    discriminator_activation_function, 11, 64, 1)
generator = CFGAN(generator_activation_function)

# Binary cross entropy: https://pytorch.org/docs/stable/nn.html?highlight=bceloss#torch.nn.BCELoss
criterion = nn.BCELoss()
# optim
generator_optim = torch.optim.Adam(
    generator.parameters(), lr=LR, betas=(0.9, 0.99))
discriminator_1_optim = torch.optim.Adam(
    discriminator_1.parameters(), lr=LR, betas=(0.9, 0.99))



In [13]:
import copy

# debug
## test for paramaters

In [323]:
data = copy.copy(dataLoader)
for real_data in data:

    # 1A: Train D1 on real
    discriminator_1.zero_grad()
    d_real_data = real_data
    # real data's lable should be true
    d_real_labe = torch.ones(d_real_data.size()[0])
    d_real_decision = discriminator_1(d_real_data.float())
    d_real_loss = criterion(
        torch.squeeze(d_real_decision), d_real_labe)
    d_real_loss.backward()

    # 1B: Train D1 on fake data
    d_fake_data = generator(torch.randn(batch, 11))
    # print(d_fake_data.size())
    d_fake_lable = torch.zeros(batch)
    d_fake_decision = discriminator_1(d_fake_data)
    d_fake_loss = criterion(torch.squeeze(
        d_fake_decision), d_fake_lable)
    d_fake_loss.backward()
    # Only optimizes D1's parameters
    discriminator_1_optim.step()

In [320]:

for i in range(100):
    data = copy.copy(dataLoader)
    for real_data in data:

        # 1A: Train D1 on real
        discriminator_1.zero_grad()
        d_real_data = real_data
        # real data's lable should be true
        d_real_labe = torch.ones(d_real_data.size()[0])
        d_real_decision = discriminator_1(d_real_data.float())
        d_real_loss = criterion(
            torch.squeeze(d_real_decision), d_real_labe)
        d_real_loss.backward()

        # 1B: Train D1 on fake data
        d_fake_data = generator(torch.randn(batch, 11))
        # print(d_fake_data.size())
        d_fake_lable = torch.zeros(batch)
        d_fake_decision = discriminator_1(d_fake_data)
        d_fake_loss = criterion(torch.squeeze(
            d_fake_decision), d_fake_lable)
        d_fake_loss.backward()
        # Only optimizes D1's parameters
        discriminator_1_optim.step()
    
    for g_index in range(64):
        # Train G on D's response
        generator.zero_grad()
        g_fake_data = generator(torch.randn(batch, 11))
        d_g_fake_decision = discriminator_1(g_fake_data)
        g_fake_lable = torch.ones(batch)
        g_loss = criterion(torch.squeeze(d_g_fake_decision), g_fake_lable)
        g_loss.backward()
        generator_optim.step()
        
    print(i,d_real_loss.tolist(),d_fake_loss.tolist(),g_loss.tolist())

0 0.01746196672320366 0.01706872694194317 0.031115896999835968
1 0.006720302160829306 0.011945049278438091 0.014585972763597965
2 0.009522311389446259 0.014734037220478058 0.019268788397312164
3 0.012144551612436771 0.009390733204782009 0.0056714885868132114
4 0.007415828760713339 0.02160409651696682 0.02024024911224842
5 0.008018492721021175 0.009443645365536213 0.007449044380337
6 0.0066832611337304115 0.0101286880671978 0.019743988290429115
7 0.00682487990707159 0.007412943057715893 0.005183602217584848
8 0.012305964715778828 0.011076987721025944 0.016665050759911537
9 0.012146513909101486 0.007454392965883017 0.008507216349244118
10 0.009904150851070881 0.01626337319612503 0.011868362314999104
11 0.0031741480343043804 0.006769614759832621 0.005074944347143173
12 0.009624160826206207 0.023218706250190735 0.020819557830691338
13 0.005209844559431076 0.0037685297429561615 0.00498428987339139
14 0.006463318597525358 0.008819002658128738 0.015797097235918045
15 0.005348532926291227 0.00

In [328]:
f = generator(torch.randn(30, 11))
print(f)
print(f.ge(0.5).long())
dis = discriminator_1(generator(torch.randn(batch, 11)))
dis

tensor([[ 1.0000, -0.0471,  0.9994,  0.9999, -0.0160,  1.0000,  0.9984,  0.9997,
          0.9998,  1.0000, -0.0273],
        [ 1.0000, -0.0421, -0.0383,  0.9999, -0.0126,  1.0000,  0.9975,  0.9997,
          0.0088,  1.0000, -0.0260],
        [ 1.0000, -0.0480,  0.9994, -0.1612, -0.0184,  0.1952,  0.9981,  0.9997,
          0.9998,  1.0000,  0.9511],
        [ 1.0000, -0.0485,  0.9994, -0.1732, -0.0192,  0.3747,  0.9984,  0.9998,
          0.9998,  1.0000, -0.0283],
        [ 1.0000,  0.9986, -0.0057,  0.9999, -0.0129,  1.0000,  0.9954,  0.9997,
          0.9998,  1.0000, -0.0269],
        [ 1.0000, -0.0470,  0.9994,  0.9994, -0.0167,  1.0000,  0.9970,  0.9997,
          0.0085,  1.0000, -0.0252],
        [ 1.0000,  0.9986,  0.9994,  0.9999, -0.0156,  1.0000,  0.9942,  0.9997,
          0.9998,  1.0000,  0.9994],
        [ 1.0000,  0.9986,  0.0486,  0.9999, -0.0130,  1.0000,  0.9984,  0.9997,
          0.0088,  1.0000, -0.0228],
        [ 1.0000,  0.9968,  0.1560,  1.0000, -0.0146,  1

tensor([[0.9873],
        [0.9902],
        [0.9868],
        [0.9911],
        [0.9828],
        [0.9841],
        [0.9899],
        [0.9935],
        [0.9879],
        [0.9876],
        [0.9885],
        [0.9862],
        [0.9885],
        [0.9669],
        [0.9879],
        [0.9904],
        [0.9818],
        [0.9885],
        [0.9900],
        [0.9911],
        [0.9716],
        [0.9901],
        [0.9890],
        [0.9899],
        [0.9854],
        [0.9891],
        [0.9903],
        [0.9823],
        [0.9891],
        [0.9884],
        [0.9718],
        [0.9874],
        [0.9873],
        [0.9871],
        [0.9886],
        [0.9861],
        [0.9883],
        [0.9903],
        [0.9905],
        [0.9902],
        [0.9904],
        [0.9789],
        [0.9875],
        [0.9930],
        [0.9880],
        [0.9905],
        [0.9879],
        [0.9881],
        [0.9908],
        [0.9846],
        [0.9817],
        [0.9875],
        [0.9901],
        [0.9883],
        [0.9904],
        [0

In [327]:
 for g_index in range(32):
    # Train G on D's response
    generator.zero_grad()
    g_fake_data = generator(torch.randn(batch, 11))
    d_g_fake_decision = discriminator_1(g_fake_data)
    g_fake_lable = torch.ones(batch)
    g_loss = criterion(torch.squeeze(d_g_fake_decision), g_fake_lable)
    g_loss.backward()
    generator_optim.step()

In [21]:
import copy
for epoch in range(num_epochs):
    # GAN1
    # 1. Train D on real+fake
    # D.zero_grad()
    data = copy.copy(dataLoader)

    for real_data in data:

        # 1A: Train D1 on real
        discriminator_1.zero_grad()
        d_real_data = real_data
        # real data's lable should be true
        d_real_labe = torch.ones(d_real_data.size()[0])
        d_real_decision = discriminator_1(d_real_data.float())
        d_real_loss = criterion(
            torch.squeeze(d_real_decision), d_real_labe)
        d_real_loss.backward()

        # 1B: Train D1 on fake data
        d_fake_data = generator(torch.randn(batch, 11))
        # print(d_fake_data.size())
        d_fake_lable = torch.zeros(batch)
        d_fake_decision = discriminator_1(d_fake_data)
        d_fake_loss = criterion(torch.squeeze(
            d_fake_decision), d_fake_lable)
        d_fake_loss.backward()
        # Only optimizes D1's parameters
        discriminator_1_optim.step()

        drl, dfl = d_real_loss.tolist(), d_fake_loss.tolist()
    for g_index in range(g_steps):
        # Train G on D's response
        generator.zero_grad()

        g_fake_data = generator(torch.randn(batch, 11))
        d_g_fake_decision = discriminator_1(g_fake_data)
        g_fake_lable = torch.ones(batch)
        g_loss = criterion(torch.squeeze(d_g_fake_decision), g_fake_lable)
        g_loss.backward()
        generator_optim.step()
        gl = g_loss.tolist()

    # GAN2
    for g_index in range(g2_steps):
        generator.zero_grad()
        noise_z = torch.randn(batch, 11)
        fake_data = generator(noise_z)
        # O = {race; native country}:(0,0) (0,1) (1,0) (1,1)
        noise_o0 = []
        noise_o1 = []
        noise_o2 = []
        noise_o3 = []

        for index, single_data in enumerate(fake_data):
            if(single_data[7] < 0.5 and single_data[9] < 0.5):
                noise_o0.append(noise_z[index].view(1, -1))
            elif(single_data[7] < 0.5 and single_data[9] >= 0.5):
                noise_o1.append(noise_z[index].view(1, -1))
            elif(single_data[7] >= 0.5 and single_data[9] < 0.5):
                noise_o2.append(noise_z[index].view(1, -1))
            else:
                noise_o3.append(noise_z[index].view(1, -1))
        ge0,ge1,ge2,ge3 = None,None,None,None
        if(len(noise_o0) != 0):
            noise_o0 = torch.cat(noise_o0)
            o0_0_lable = generator(noise_o0, 0)[:, -1]
            o0_1_lable = generator(noise_o0, 1)[:, -1].detach()
            g_error0 = criterion(o0_0_lable, o0_1_lable)
            g_error0.backward()
            ge0 = g_error0.tolist()
        if(len(noise_o1) != 0):
            noise_o1 = torch.cat(noise_o1)
            o1_0_lable = generator(noise_o1, 0)[:, -1]
            o1_1_lable = generator(noise_o1, 1)[:, -1].detach()
            g_error1 = criterion(o1_0_lable, o1_1_lable)
            g_error1.backward()
            ge1 = g_error1.tolist()
        if(len(noise_o2) != 0):
            noise_o2 = torch.cat(noise_o2)
            o2_0_lable = generator(noise_o2, 0)[:, -1]
            o2_1_lable = generator(noise_o2, 1)[:, -1].detach()
            g_error2 = criterion(o2_0_lable, o2_1_lable)
            g_error2.backward()
            ge2 = g_error2.tolist()
        if(len(noise_o3) != 0):
            noise_o3 = torch.cat(noise_o3)
            o3_0_lable = generator(noise_o3, 0)[:, -1]
            o3_1_lable = generator(noise_o3, 1)[:, -1].detach()
            g_error3 = criterion(o3_0_lable, o3_1_lable)
            g_error3.backward()
            ge3 = g_error3.tolist()

        generator_optim.step()

    if epoch % print_interval == 0:
        print("Epoch %s: D (%s real_err, %s fake_err) G_l (%s err) G_0l (%s) G_1l (%s) G_2l (%s) G_3l (%s);" % (
            epoch, drl, dfl, gl, ge0, ge1, ge2, ge3))

Epoch 0: D (0.22125402092933655 real_err, 0.059217192232608795 fake_err) G_l (0.015695534646511078 err) G_0l (None) G_1l (None) G_2l (0.07190127670764923) G_3l (0.07072243094444275);
Epoch 1: D (0.8784480094909668 real_err, 0.14986123144626617 fake_err) G_l (0.05311468243598938 err) G_0l (None) G_1l (None) G_2l (None) G_3l (0.04634556174278259);
Epoch 2: D (0.32248008251190186 real_err, 0.18921847641468048 fake_err) G_l (0.010749738663434982 err) G_0l (None) G_1l (None) G_2l (None) G_3l (0.06491663306951523);
Epoch 3: D (0.2830143868923187 real_err, 0.25805848836898804 fake_err) G_l (0.02052653394639492 err) G_0l (None) G_1l (None) G_2l (None) G_3l (0.029447432607412338);
Epoch 4: D (0.08148318529129028 real_err, 0.15486261248588562 fake_err) G_l (0.0037686177529394627 err) G_0l (None) G_1l (None) G_2l (None) G_3l (0.09675496816635132);
Epoch 5: D (0.6273917555809021 real_err, 0.5055474638938904 fake_err) G_l (0.02917487919330597 err) G_0l (None) G_1l (None) G_2l (None) G_3l (0.0302715

KeyboardInterrupt: 